# Library Database Management - Solutions

This notebook contains solutions for the library database management exercises. We'll be working with bibliographic resources and publisher data to create a relational database.

## Setup

First, let's import the necessary libraries:

In [ ]:
# Import required libraries
import pandas as pd
import sqlite3
import os

## Exercise 1: Creating the DataFrames

In this exercise, we'll read the CSV files and create DataFrames with unique IDs for each row.

In [ ]:
# Step 1: Read the data.csv file with read_csv() function
bibliographic_df = pd.read_csv('/path/to/your/folder/data.csv')

# Step 2: Read the publishers.csv file
publishers_df = pd.read_csv('/path/to/your/folder/publisher.csv')

# You can decide to use .head() or not to display the first 5 rows of the DataFrame
# printing a string like "Bibliographic Resources DataFrame:" before the DataFrame will make it easier to understand the output
print("Bibliographic Resources DataFrame:")
print(bibliographic_df.head())
print("\nPublishers DataFrame:")
print(publishers_df.head())

In [ ]:
# Step 3 & 4: Add unique identifiers to each DataFrame

# For bibliographic_df we create a new column called 'unique_id' and assign a unique identifier to each row
# We use a for loop to iterate over the length of the DataFrame and assign a unique identifier to each row
# The unique identifier is created by concatenating the string 'biblio-' with the row index
# We use the range() function to generate a sequence of numbers from 0 to the length of the DataFrame
# We convert the numbers to strings using the str() function and concatenate them with the string 'biblio-'
# The resulting unique identifiers are stored in the 'unique_id' column of the DataFrame

bibliographic_df['unique_id'] = ['biblio-' + str(i) for i in range(len(bibliographic_df))]

# We repeat the same process for the publishers_df DataFrame, creating a new column called 'unique_id' and assigning unique identifiers to each row
publishers_df['unique_id'] = ['publisher-' + str(i) for i in range(len(publishers_df))]

# Step 5: Print the first five rows of each DataFrame with unique IDs
print("Bibliographic Resources DataFrame with unique IDs:")
print(bibliographic_df.head())
print("\nPublishers DataFrame with unique IDs:")
print(publishers_df.head())

## Exercise 2: Merging DataFrames

Now, we'll merge the two DataFrames to maintain the relationship between resources and publishers.

In [ ]:
# Step 1: Merge the DataFrames on the publisher id field
# The 'publisher' column in bibliographic_df corresponds to the 'id' column in publishers_df
# We use the pd.merge() function to merge the two DataFrames based on these columns
# The 'how' parameter is set to 'left' to keep all rows from the bibliographic_df DataFrame
# The resulting merged DataFrame contains the columns from both DataFrames, with the 'publisher' column replaced by the 'name' column from publishers_df
merged_df = pd.merge(
    bibliographic_df,
    publishers_df,
    left_on='publisher',  # Column in bibliographic_df
    right_on='id',        # Column in publishers_df
    how='left'            # Keep all rows from bibliographic_df
)

# Step 2: Rename columns to avoid confusion
# We rename the columns to avoid confusion, as both DataFrames have an 'id' column
# We rename the 'id_x' column to 'bibliographic_id' and the 'id_y' column to 'publisher_id'
merged_df = merged_df.rename(columns={
    'id_x': 'bibliographic_id',  # From bibliographic_df
    'id_y': 'publisher_id',      # From publishers_df
    'name': 'publisher_name'     # To clarify this is the publisher's name
})

# Step 3: Print the first five rows of the newly created merged DataFrame
print("Merged DataFrame:")
print(merged_df.head())

## Exercise 3: Connecting to SQLite and Creating the Database

Finally, we'll create a SQLite database with appropriate tables based on our DataFrames and the UML diagram.

In [ ]:
# Step 1: Establish a connection to a new SQLite database
# checking if the os path exists is just to make sure that the database file does not already exist
# if it does, we remove it to start fresh
if os.path.exists('library.db'):
    os.remove('library.db')

# creating a new connection to the SQLite database
connection = sqlite3.connect('library.db')
# creating a cursor object to execute SQL queries and operate on the db
cursor = connection.cursor()

# Step 2: Create the Publisher table
# The Publisher table should be created first because BibliographicResource will reference it
# Check SQL documentation for more information on the data types and operators used.
# Note that the capitalization ("TEXT" vs "text") is not required, but it is a common convention to use uppercase for SQL keywords
# to distinguish them from table and column names.
# "--" is used for comments in SQL queries, this allows you to document your queries for future reference.
cursor.execute('''
CREATE TABLE Publisher (
    id TEXT PRIMARY KEY,          -- Original ID from publishers.csv (e.g., 'crossref:15')
    name TEXT NOT NULL,           -- Publisher's name
    unique_id TEXT NOT NULL       -- Our manually created unique ID
);
''')

# Step 3: Create the BibliographicResource table
cursor.execute('''
CREATE TABLE BibliographicResource (
    id TEXT PRIMARY KEY,                -- Original ID from data.csv (e.g., 'doi:10.1037/e383822004-001')
    title TEXT NOT NULL,                -- Title of the resource
    type TEXT,                          -- Type of the resource (e.g., 'dataset')
    publisher TEXT,                     -- Foreign key referencing Publisher.id
    unique_id TEXT NOT NULL,            -- Our manually created unique ID
    FOREIGN KEY (publisher) REFERENCES Publisher(id)
);
''')

# Step 4: Insert the Publisher data
# We use a for loop to iterate over the rows of the publishers_df DataFrame
# For each row, we execute an INSERT query to add the data to the Publisher table
for index, row in publishers_df.iterrows():
    cursor.execute('''
    INSERT INTO Publisher (id, name, unique_id)
    VALUES (?, ?, ?);
    ''', (row['id'], row['name'], row['unique_id']))

# Step 5: Insert the BibliographicResource data
for index, row in bibliographic_df.iterrows():
    cursor.execute('''
    INSERT INTO BibliographicResource (id, title, type, publisher, unique_id)
    VALUES (?, ?, ?, ?, ?);
    ''', (row['id'], row['title'], row['type'], row['publisher'], row['unique_id']))

# Commit the changes and close the connection
# It is important to commit the changes to the database after inserting data
# This ensures that the changes are saved permanently to the database
# (and this is why in this script we check if the file is already existing and remove it to start fresh)
# Note that you have to comment that part of the script if you want to access to you db file later
connection.commit()

# Finally we close the connection
connection.close()

# It is a good practice to add a final print statement with something like "job done" or "database created successfully"
# This will let you know that the script has completed successfully
print("Database 'library.db' created successfully with Publisher and BibliographicResource tables.")

## Verification

Let's verify that our database was created correctly by querying some data from it.

In [ ]:
# Connect to the database previously created
connection = sqlite3.connect('library.db')
# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Query the Publisher table
print("Publishers:")
# Here again the "*" means "everything" so we are selecting all the columns from the Publisher table
# the "LIMIT 5" is used to limit the number of rows returned to 5
# the "cursor.fetchall()" method is used to retrieve all the rows returned by the query
# and then we iterate over them to print each row
cursor.execute("SELECT * FROM Publisher LIMIT 5;")
for row in cursor.fetchall():
    print(row)

# Query the BibliographicResource table
print("\nBibliographic Resources:")
cursor.execute("SELECT * FROM BibliographicResource LIMIT 5;")
for row in cursor.fetchall():
    print(row)

# Query to join the tables (similar to our merged DataFrame)
print("\nJoined Data:")
# This query is slightly more complex as it involves joining the Publisher and BibliographicResource tables
# We use the LEFT JOIN clause to keep all rows from the BibliographicResource table
# We also use the AS keyword to alias the column names for clarity
# The LIMIT 5 clause limits the number of rows returned to 5
cursor.execute("""
SELECT b.id, b.title, b.type, p.name as publisher_name
FROM BibliographicResource b
LEFT JOIN Publisher p ON b.publisher = p.id
LIMIT 5;
""")
for row in cursor.fetchall():
    print(row)

# Close the connection
connection.close()